In [1]:
import yaml 
import requests
import pandas as pd
import sqlite3
from tqdm import tqdm
import os

In [2]:
conn = sqlite3.connect("data/database.db")

In [3]:
tickers = pd.read_sql_query("SELECT Distinct ticker FROM master_ticker", conn)["ticker"].tolist()

In [4]:
with open("keys.yaml") as keys:
    try:
        api_keys = yaml.safe_load(keys)
    except yaml.YAMLError as exc:
        print(exc)

In [5]:
def get_financials_data(tickers_list, save_dir):
    #Get files in dir
    file_list = os.listdir(save_dir)
    file_list = [f.split('.')[0] for f in file_list]
    gen = (t for t in tqdm(tickers_list) if t not in file_list)

    for t in gen:
        try:
                financials = requests.get("https://financialmodelingprep.com/api/v3/income-statement/" + t +"?period=annual&apikey=" + api_keys["fsapikey"])
                df_ticker = pd.DataFrame(financials.json())
                df_ticker.to_csv("data/income_statements/" + t + ".csv", index = False)
        except:
            print("Fail for ticker ... " + t)
            break

In [6]:
get_financials_data(tickers, "data/income_statements")

100%|██████████| 503/503 [00:07<00:00, 66.32it/s] 
